# 

Importar librerias

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from src import check_ids, get_connection, close_connection, insert_query
from src.teradata_query_batches import fetch_data_in_batches
from teradataml import execute_sql
import pandas as pd
import joblib
import math
from teradataml.dataframe.fastload import fastload

In [12]:
batch_size = 500000
query_file = '../sql/queries/customerid_query.sql'
columns = ['UNIDAD DE NEGOCIO', 'ZONA','REGION','CUSTOMER_ID', 'LLAVE','MIN FECHA','MAX FECHA','TRANSACTIONS','MONEY','FECHAS','VOLUME','QTY']

In [13]:
get_connection()

Conexión a Teradata Vantage establecida correctamente.


In [14]:
preprocessor = joblib.load('../models/preprocessor.pkl')
model = joblib.load('../models/random_forest_model.pkl')

In [15]:
all_data = pd.DataFrame()

with open(query_file, 'r') as file:
    query_template = file.read()

In [16]:
count_query = """
    SELECT COUNT(*) AS total_records
    FROM COL_MKT.INFO_PER_CUSTOMER
    WHERE CUSTOMER_ID IS NOT NULL
    """

total_records = pd.DataFrame(execute_sql(count_query))[0][0]
total_batches = math.ceil(total_records / batch_size)
print(total_batches)

89


In [ ]:
for batch in range(total_batches):
    print(batch)
    start = batch * batch_size + 1
    end = start + batch_size - 1
    query = query_template.format(start=start, end=end)
    try:
        
        result = execute_sql(query)
        
        batch_df = pd.DataFrame(result, columns=columns)
        batch_df['VOLUME'] = pd.to_numeric(batch_df['VOLUME'], errors='coerce')
        batch_df['QTY'] = pd.to_numeric(batch_df['QTY'], errors='coerce')
        batch_df['MONEY'] = pd.to_numeric(batch_df['MONEY'], errors='coerce')
        batch_df['MIN FECHA'] = pd.to_datetime(batch_df['MIN FECHA'], format='%Y-%m-%d')
        batch_df['MAX FECHA'] = pd.to_datetime(batch_df['MAX FECHA'], format='%Y-%m-%d')
        batch_df[['ID_LENGTH', 'ID_HAS_REPEATED_DIGITS', 'ID_IS_ASCENDING', 'ID_IS_DESCENDING', 'ID_HAS_REPETITIVE_PATTERN']] = batch_df[
            'CUSTOMER_ID'].apply(lambda x: pd.Series(check_ids(x)))
        
        input_processed = preprocessor.transform(batch_df)
        prediction = model.predict(input_processed)
        
        batch_df['IS_VALID'] = prediction
        batch_df = batch_df[['UNIDAD DE NEGOCIO','ZONA','REGION','CUSTOMER_ID','IS_VALID']]
        
        print('Insertando en Teradata...')
        
        fastload(df=batch_df, schema_name='COL_MKT', table_name='CUSTOMER_PREDICTIONS_AUX',if_exists='replace')
        
        print('Inserción auxiliar hecha')
        
        insert_query_predictions = """
            INSERT INTO COL_MKT.CUSTOMER_PREDICTIONS
            SELECT * FROM COL_MKT.CUSTOMER_PREDICTIONS_AUX;
            """
        
        
        execute_sql(insert_query_predictions)
        
        print(f'Inserción hecha de {batch_df.shape[0]}')
        
        all_data = pd.concat([all_data, batch_df], ignore_index=True)
        
    except Exception as e:
        print(f'Error al ejecutar la consulta para el lote {batch}: {e}')

0


In [9]:
close_connection()